## Business Use Cases DEMO

In [ ]:
### Most popular region
SELECT regioncity, COUNT(1) AS hit_count
FROM access_log 
    JOIN geo_base ON(access_log.host = geo_base.host)
GROUP BY regioncity ORDER BY hit_count LIMIT 100
;

In [ ]:
### Real users vs bots distribution
SELECT request,
    SUM(IF(robot.bot_name IS NULL, 1, 0)) AS user_hit_count,
    SUM(IF(robot.bot_name IS NULL, 1, 0)) AS user_hit_count
FROM access_log LEFT OUTER 
    JOIN robot ON(
        access_log.host = robot.host AND
        access_log.user_agent = robot.user_agent
    )
GROUP BY request
;

In [ ]:
### Male vs female audience (per region)
SELECT regioncity,
    SUM(IF(user.gender = "M", 1, 0)) AS male_hit_count,
    SUM(IF(user.gender = "F", 1, 0)) AS female_hit_count
FROM access_log
    JOIN geo_base ON (access_log.host = geo_base.host)
    JOIN user ON(
        access_log.host = user.host AND
        access_log.user_agent = user.user_agent
    )
;

In [ ]:
### Average customer age
SELECT region, AVG(user.age)
FROM access_log
    JOIN geo_base ON(access_log.host = geo_base.host)
    JOIN user ON(
        access_log.host = user.host
        access_log.user_agent = user.user_agent
    )
GROUP BY region
;

## SQL

In [ ]:
# Basic
SELECT *
FROM Customers
    WHERE Country='Mexico'AND PostalCode='05023';

In [ ]:
# Order by 
SELECT *
FROM Customers
ORDER BY Country
;

SELECT *
FROM Customers
ORDER BY Country DESC
;

In [ ]:
# Join
SELECT 
    Orders.OrderID,
    Customers.CustomerName,
    Orders.OrderDate
FROM Orders
    JOIN Customers ON Orders.CustomerID = Customers.CustomerID
;

In [ ]:
# Aggregation
SELECT <column_name>, <agg_functions>(column_name)
FROM <table_name>
GROUP BY <column_name>
;


SELECT Country, COUNT(*)
FROM Customers
GROUP BY Country
;
SELECT Country, State, COUNT(*)
FROM Customers
GROUP BY Country, State
;
SELECT OrderDate, MIN(OrderID), MAX(OrderID)
FROM Orders
GROUP BY OrderDate
;

In [ ]:
# Where: filter data before aggregation
# Having: filter data after aggregation (more resource consumption)
SELECT <column_name>, <agg_functions>(column_name)
FROM <table_name>
    WHERE <column_name> <operator> <value>
GROUP BY <column_name>
    HAVING <agg_functions>(column_name)
;

SELECT Country
FROM Cusomers
GROUP BY Country
    HAVING COUNT(*) > 10
;

In [ ]:
# Subqueries
SELECT *
FROM Customers
    WHERE Country IN(
        SELECT Country
        FROM Customers
        GROUP BY Country
            HAVING COUNT(*) > 10
    )
;

## Hive Data Definition Language (DDL)

In [ ]:
# Create table
CREATE TABLE my_table_name(
    dummy_column STRING,
    another_column STRING
);

In [ ]:
# Describe database: check HDFS location
DESCRIBE DATABASE default;
DESCRIBE TABLE[EXTENDED|FORMATTED] <table_name>;

In [ ]:
# Create table from an existing HDFS dataset
USE <database_name>;
CREATE TABLE <table_name>(
    <column_name> <column_type>,
    ...
)
LOCATION "path/to/hdfs"
;

In [ ]:
# default table = managed table: drop table = metadata + data in HDFS removed
# external table: drop table = metadata removed, data in HDFS unchanged
# temporary table: auto removed after a hive session closed
%% tab_dataset/part-*
first   line    1
second  line    3
last    line    5
##############################################
USE mydb;
DROP TABLE IF EXIST tab_dataset;
CREATE EXTERNAL TABLE tab_dataset(
    first_column STRING,
    second_column STRING,
    value INT
)
ROW FORMAT delimited
    fields terminated by '\t'
LOCATION 'path/to/tab_dataset'
;

In [ ]:
# Complex type
%% dataset/part-*
John Doe'^A'10000.0'^A'Mary Smith'^B'Todd Jones'^A'Federal Taxes'^C'.2'^B'State Taxes'^c'.05'^B'Insurance'^C'.1'^A'Michigan Avenue.'^B'Chicago'^B'IL'^B'60600
############################################
CREATE TABLE employees(
    name STRING,
    salary FLOAT,
    subordinates ARRAY<STRING>,
    deductions MAP<STRING, FLOAT>,
    address STRUCT<street:STRING, city:STRING, state:STRING, zip:INT>
)
ROW FORMAT delimited
    fields terminated by '\001' #^A
    collection items terminated by '\002' #^B
    map keys terminated by '\003' #^C
    lines terminated by '\n'
STORED AS <file_format>
LOCATION 'path/to/dataset'
;

## Hive Data Manipulation Language(DML)

In [ ]:
# DML import from HDFS -> Hive
LOAD DATA INPATH '/path/employees-data'
INTO TABLE employees
;

# DML import from local FS -> Hive
LOAD DATA LOCAL INPATH '/path/employees-data'
INTO TABLE employees
;

# Replace data in tables
LOAD DATA [LOCAL] INPATH '/path/employees-data'
OVERWRITE INTO TABLE employees
;

In [ ]:
# DML export Hive -> local or HDFS
INSERT OVERWRITE [LOCAL] DIRECTORY 'tmp/employees'
SELECT name, salary, address
FROM employees
    WHERE ...
;

# DML export multiple insert
INSERT OVERWRITE [LOCAL] DIRECTORY 'tmp/ca_employees'
SELECT name, salary, address
FROM employees
    WHERE state = 'CA'
INSERT OVERWRITE [LOCAL] DIRECTORY 'tmp/ny_employees'
SELECT name, salary, address
FROM employees
    WHERE state = 'NY'
;

In [ ]:
# move tables -> tables
FROM raw_table
INSERT OVERWRITE TABLE us_employees
SELECT *
WHERE raw_table.country = 'US'
INSERT OVERWRITE TABLE uk_employees
SELECT *
WHERE raw_table.country = 'UK'
...
;

# Create tables from tables
CREATE TABLE ca_employees
AS SELECT name, salary, address
FROM employees
WHERE state = 'CA'
;

## Hive Queries

In [ ]:
SELECT ... FROM (<-- MAP)
WHERE (<-- MAP)
GROUP BY (<-- Shuffle and Sort)
HAVING (<-- Reduce)
JOIN (<-- Map, Reduce)
ORDER BY / SORT BY (<-- Reduce)

## Parsing data

In [ ]:
%%log
127.0.0.1 - frank [10/Oct/2000:13:55:36 -0700] "GET /apache_pb.gif HTTP/1.0" 200
123.65.150.10 - - [23/Aug/2010:03:50:59 +0000] "POST /wordpress3/wp-admin/admin-ajax.php HTTP/1.1" 200
#######################################
CREATE EXTERNAL TABLE apache_log(
    ip STRING,
    auth_unused STRING,
    auth_user STRING,
    request_time STRING,
    request STRING,
    status_code INT
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '^(\\S*) (\\S*) (\\S*) \\[([^\\]]*)\\] "([^"]*)" (\\S*) .*$' # string,string,string,[string],"string",string
    )
LOCATION 'path/to/log'
;

## View - Read only Table

In [ ]:
# create view table from an existing table
CREATE VIEW apache_log_view(
    ip,
    request_year,
    request,
    status_code
)
AS SELECT
    ip,
    regexp_extract(request_time, "\\d+\\/\\w+\\/(\\d+)", 1), # 10/Oct/2000:13:55:36-0700 -> 2000
    request,
    status_code
FROM apache_log
;

In [7]:
# test
import re
re.search('\\d+\\/\\w+\\/(\\d+)', '10/Oct/2000:13:55:36-0700').group(1)

'2000'

## Hive Analytics (UDF, UDAF, UDTF)

In [ ]:
1. Operators
    =, !=, <, >, IS NULL, +, - , *, /, AND, OR, IN ...

In [ ]:
2. Functions (UDFs = User Defined Functions)   # map 1 -> 1
    math: round, floor, ceil, exp, log...
    date: to_date, from_unixtimestamp, year...
    conditional: if, isnull, case...
    string: char, concat, lower, trim, repeat...

In [ ]:
3. Aggregate functions (UDAFs) # map n -> 1
    count, sum, min, max, corr...

In [ ]:
4. Table-generating functions (UDTFs) # map 1->n
    explode, posexplode, parse_url_tuple...

## Hive Streaming

In [ ]:
# Word counts by Hive
ADD FILE /path/to/mapper.py
ADD FILE /path/to/reducer.py

FROM(
    FROM wikipedia_sample
    SELECT TRANSFORM(line)
    USING "./mapper.py" AS word, counts
    DISTRIBUTE BY word SORT BY word
) word_pairs
SELECT TRANSFORM(word_pairs.word, word_pairs.counts)
USING "./reducer.py"
AS word, counts


## Hive Partitioned Table Functions (PTF)

In [ ]:
# Original
FROM transactions
    SELECT TRANSFORM(customerID, change)
    USING "./locate_overdraft.py"
DISTRIBUTE BY customerID
SORT BY customerID, timestamp

# PTF function map n->1
SELECT
    customerID,
    transactionID,
    change,
    SUM(change) OVER(
        PARTITION BY customerID
        ORDER BY transactionID
    )
FROM transactions
SORT BY customerID, transactionID

## Hive Optimization

In [ ]:
# Original
SELECT region, COUNT(1) AS hit_count
FROM access_log 
    JOIN geo_base ON(access_log.host = geo_base.host)
WHERE access_log.datetime BETWEEN "recent_date" AND "now"
GROUP BY region
ORDER BY hit_count LIMIT 100


Rules:
    1. partitioned columns stay at the ends of SELECT
    2. partitioned columns must be specified in order
    3. Use configuration parameters
    4. Control empty partitions

In [ ]:
# Partition data: reduce the computational data processing
# create table
CREATE TABLE partitioned_access_log(
    ip STRING,
    ...
)
PARTITIONED BY(year STRING, month STRING, day STRING)
...
;

# transfer data
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.error.on.empty.partition=true;

FROM raw_access_log
INSERT OVERWRITE TABLE partitioned_access_log
PARTITION(year="2017", month="03", day="25")
SELECT ip, ..., year, month, day

In [ ]:
# Buckets: the number of part-* files in HDFS folder
CREATE TABLE granular_access_log(
    ip STRING,
    ...
)
PARTITIONED BY(request_date STRING)
CLUSTERED BY(user_id, ...) SORTED BY(user_id) INTO 200 BUCKETS
...
;

# Sampling: process x% of all buckets
SELECT ip,...
FROM granular_access_log
    TABLESAMPLE(BUCKET 1 OUT OF 4 ON user_id) # 25%
...
;

In [ ]:
# Template 1
SET mapred.reduce.tasks = 200;

FROM raw_access_log
INSERT OVERWRITE TABLE granular_access_log
PARTITION BY(request_date)
SELECT ..., request_date
WHERE ...
DISTRIBUTE BY user_id
[SORT BY user_id]
;

# Template 2
SET hive.enforce.bucketing = true;

FROM raw_access_log
INSERT OVERWRITE TABLE granular_access_log
PARTITION BY(request_date)
SELECT ..., request_date
WHERE ...
;